# DLE303 Assessment 3 - Main
## Rachel Tucker & Michael Cuffe

## Dependencies:
- Python 3.12

In [41]:
!pip install transformers torch pandas numpy matplotlib seaborn > null 2>&1

# import libraries

In [42]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd

# This is where we split the data into training and testing sets 

In [43]:
import pandas as pd

# Load the dataset
data = pd.read_csv('data/200k_Short_Texts.csv')
data = data.dropna()
data = data.sample(frac=1).reset_index(drop=True)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenize the Data

In [44]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')

def tokenize_data(data):
    return tokenizer(
        data['text'].tolist(),
        padding=True,
        truncation=True,
        return_tensors='pt',
    )

train_encodings = tokenize_data(train_data)
test_encodings = tokenize_data(test_data)

C:\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


# Create a Dataset class

In [45]:
import torch

class HumorDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).clone().detach()
        return item

    def __len__(self):
        return len(self.labels)


# Flatten\ing (Kind of)

In [ ]:
train_labels = train_data['humor'].astype(int).tolist()
test_labels = test_data['humor'].astype(int).tolist()

train_dataset = HumorDataset(train_encodings, train_labels)
test_dataset = HumorDataset(test_encodings, test_labels)

# Define the Model

In [46]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the Model

In [47]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

C:\Users\mikea\AppData\Local\Temp\ipykernel_2112\3297802665.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


ValueError: Target size (torch.Size([16])) must be the same as input size (torch.Size([16, 2]))